In [1]:
# Import libraries

import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler


In [2]:
# 1. Load the dataset and explore the variables.
churnData = pd.read_csv (r'\Users\marcp\Desktop\Ironhack\Clases\Week 4\Day 1\4. Labs\customer_churn.csv')

In [3]:
# 1. We explore, and see the headers

churnData.head()

customerID  gender  SeniorCitizen Partner Dependents  tenure PhoneService  \
0  7590-VHVEG  Female              0     Yes         No       1           No   
1  5575-GNVDE    Male              0      No         No      34          Yes   
2  3668-QPYBK    Male              0      No         No       2          Yes   
3  7795-CFOCW    Male              0      No         No      45           No   
4  9237-HQITU  Female              0      No         No       2          Yes   

      MultipleLines InternetService OnlineSecurity  ... DeviceProtection  \
0  No phone service             DSL             No  ...               No   
1                No             DSL            Yes  ...              Yes   
2                No             DSL            Yes  ...               No   
3  No phone service             DSL            Yes  ...              Yes   
4                No     Fiber optic             No  ...               No   

  TechSupport StreamingTV StreamingMovies        Contract PaperlessBilling  \
0          No          No              No  Month-to-month              Yes   
1          No          No              No        One year               No   
2          No          No              No  Month-to-month              Yes   
3         Yes          No              No        One year               No   
4          No          No              No  Month-to-month              Yes   

               PaymentMethod MonthlyCharges  TotalCharges Churn  
0           Electronic check          29.85         29.85    No  
1               Mailed check          56.95        1889.5    No  
2               Mailed check          53.85        108.15   Yes  
3  Bank transfer (automatic)          42.30       1840.75    No  
4           Electronic check          70.70        151.65   Yes  

[5 rows x 21 columns]

In [4]:
# 2. We will try to predict variable Churn using a logistic regression on variables tenure, SeniorCitizen,MonthlyCharges.
# You determine Churn as your Y, and the others as your x, we'll have to convert themli them and scalate
# 3. Extract the target variable.
# 4. Extract the independent variables and scale them.

X = churnData[['tenure', 'SeniorCitizen','MonthlyCharges']] # Define the independent variables
y = (churnData.Churn == 'Yes').astype(int) # define the dependent variables and change it typ
transformer = StandardScaler().fit(X) # you define a variable with the X, but normalized
scaled_x = transformer.transform(X) # you transform the variables


In [6]:
# 5. Now we are ready to build the model:

model = LogisticRegression(random_state=0, solver='lbfgs',
                    multi_class='ovr').fit(scaled_x, y)

# We define the model 

In [8]:
# 6. Evaluate the model

model.score(scaled_x, y)

# 7. Why is it predictive ? As our data is imbalanced (most of the data belongs to a cathegory), it will always predicts most
# of it although classifying the other bad

0.7911401391452506

In [9]:
from sklearn.metrics import confusion_matrix

In [11]:
# 8. Apply SMOTE technique and see if there is any improvement:

from imblearn.over_sampling import SMOTE
smote = SMOTE()
X_sm, y_sm = smote.fit_sample(scaled_x, y)
y_sm.value_counts()


1    5174
0    5174
Name: Churn, dtype: int64

In [12]:
## model witg SMOTE
model = LogisticRegression(random_state=0, solver='lbfgs',
                        multi_class='ovr').fit(X_sm, y_sm)


In [13]:
# 8. Checking the improvement: Has dicreased
model.score(scaled_x, y)

0.7302285957688485

In [14]:
# We can evaluate with the confusion matrix
from sklearn.metrics import confusion_matrix

y_pred=model.predict(X_sm)
confusion_matrix(y_pred, y_sm)

y_pred=model.predict(scaled_x)
confusion_matrix(y_pred, y)

array([[3765,  491],
       [1409, 1378]], dtype=int64)

In [15]:
# 9. Applying the under sampling

from imblearn.under_sampling import TomekLinks

tl = TomekLinks('majority')
X_tl, y_tl = tl.fit_sample(scaled_x, y)
y_tl.value_counts()

C:\Users\marcp\anaconda3\lib\site-packages\imblearn\utils\_validation.py:635: FutureWarning: Pass sampling_strategy=majority as keyword args. From version 0.9 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.9 "


0    4694
1    1869
Name: Churn, dtype: int64

In [16]:
model = LogisticRegression(random_state=0, solver='lbfgs',
                        multi_class='ovr').fit(X_tl, y_tl)


In [17]:
model.score(X_tl, y_tl)
y_pred=model.predict(X_tl)
confusion_matrix(y_pred, y_tl)

model.score(scaled_x, y)
y_pred=model.predict(scaled_x)
confusion_matrix(y_pred, y)

array([[4515,  891],
       [ 659,  978]], dtype=int64)

In [18]:
# It imporves the model:

model.score (X_tl, y_tl)

0.7924729544415664